<font size=50 color=darkblue>Min Cost Flow LP</font>

# Problem modelling with DOcplex

## Import necessary modules

- DOcplex will be used to model and solve the Min-Cost-Flow LP
- Import module `itertools` to group links based on their tails/heads

In [ ]:
from docplex.mp.model import Model
import itertools

## Min-Cost-Flow LP <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\sum_{(u,v)\in \mathcal{E}}c_{(u,v)}\cdot \color{blue}x_{(u,v)}$$
**Subject to**
### \begin{align*}
\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} = 0,\qquad&\forall v\in \mathcal{V}\backslash\left({\color{green}\mathcal{S}}\bigcup {\color{red}\mathcal{D}}\right)\\
\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} \ge d_{(v)},\qquad&\forall v\in {\color{red}\mathcal{D}}\\
\sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} - \sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} \le s_{(v)},\qquad&\forall v\in {\color{green}\mathcal{S}}\\
0\le {\color{blue}x_{(u,v)}}\le \mu_{(u,v)},\qquad&\forall(u,v)\in \mathcal{E}
\end{align*}

## Display the network
<img src='img/mcf.png' width=700></img>

## Define the model input
### \begin{align*}
V =\,& \{0, 1, 2, 3, 4, 5, 6, 7, 8\}&\\
E =\,& \{(0,3), (0,4), (1,3), (1,4), (2,3), (2,4), (3,4), (3,5), (3,6), (4,7), (4,8)\}&\\
{\color{green}S} =\,& \{0, 1, 2\}&\\
{\color{red}D} =\,& \{5, 6, 7, 8\}&
\end{align*}

In [ ]:
V = [0, 1, 2, 3, 4, 5, 6, 7, 8]
E = [(0,3), (0,4), (1,3), (1,4), (2,3), (2,4), (3,4), (3,5), (3,6), (4,7), (4,8)]
S = [0, 1, 2]
D = [5, 6, 7, 8]

## Define in-links <code><font color='red'>delta_min</font></code> and out-links <code><font color='green'>delta_pls</font></code>, subscriptable by node indices
### \begin{align*}
{\color{red}\delta}^{\color{red}-}_{(v)} = \{e\in \mathcal{E}\,|&\,\text{head}(e)\equiv v \},&\quad\forall v\in \mathcal{V}\\
{\color{green}\delta}^{\color{green}+}_{(v)} = \{e\in \mathcal{E}\,|&\,\text{tail}(e)\equiv v \},&\quad\forall v\in \mathcal{V}\\
\end{align*}

In [ ]:
# Anonymous functions
head = lambda e: e[1]
tail = lambda e: e[0]
delta_min = {v: set(delt_min) for v, delt_min in itertools.groupby(sorted(E, key=head), head)}
delta_pls = {v: set(delt_pls) for v, delt_pls in itertools.groupby(sorted(E, key=tail), tail)}

## Define supply capacities $s_{(v)},\,\forall v\in {\color{green}\mathcal{S}}$ and demands $d_{(v)},\,\forall v\in {\color{red}\mathcal{D}}$, subscriptable by node indices

In [ ]:
# Supply capacities, subscriptable by node indices
s = dict(zip(S, [100, 200, 300]))

# Demands, subscriptable by node indices
d = dict(zip(D, [150, 150, 150, 150]))

## Define unit flow costs $c_{(e)},\,\forall e\in \mathcal{E}$ and link capacities $\mu_{(e)},\,\forall e\in \mathcal{E}$, subscriptable by link indices

In [ ]:
# Unit flow costs, subscriptable by link indices
c = dict(zip(E, [0.8, 2.0, 2.5, 1.0, 1.2, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0]))

# Capacities, subscriptable by link indices
mu = dict(zip(E, [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]))

## Create the LP model for Min-Cost-Flow problem using DOcplex

In [ ]:
mcf_LP = Model(name='Min Cost Flow')

## Define decision variables: $0\le {\color{blue}x_{(e)}}\le\mu_{(e)},\quad\forall e\in \mathcal{E}$

In [ ]:
mcf_LP.clear() # This line is optional. Its purpose is to avoid variable duplicates
x = {(u,v): mcf_LP.continuous_var(name=f'x({u},{v})', ub=mu[u,v]) for u,v in E}

## Define flow conservation constraints, subscriptable by node indices $$\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} = 0,\quad\forall v\in \mathcal{V}\backslash\left({\color{green}\mathcal{S}}\cup {\color{red}\mathcal{D}}\right)$$

In [ ]:
flow_conserv_ct = {v: mcf_LP.add_constraint(ct=mcf_LP.sum(x[e] for e in delta_min[v]) - mcf_LP.sum(x[e] for e in delta_pls[v]) == 0,
                                            ctname=f'flow_conservation({v})') for v in set(V) - set(S + D)}

## Define demand satisfaction constraints, subscriptable by node indices $$\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} \ge d_{(v)},\quad\forall v\in {\color{red}\mathcal{D}}$$

In [ ]:
demand_sat_ct = {v: mcf_LP.add_constraint(ct=mcf_LP.sum(x[e] for e in delta_min[v]) - mcf_LP.sum(x[e] for e in delta_pls.get(v, [])) >= d[v],
                                          ctname=f'demand_satisfaction({v})') for v in D}

## Define supply capacity constraints, subscriptable by node indices $$\sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} - \sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} \le s_{(v)},\quad\forall v\in {\color{green}\mathcal{S}}$$

In [ ]:
sup_cap_ct = {v: mcf_LP.add_constraint(ct=mcf_LP.sum(x[e] for e in delta_pls[v]) - mcf_LP.sum(x[e] for e in delta_min.get(v, [])) <= s[v],
                                       ctname=f'capacity_supply({v})') for v in S}

## Define objective function $$\textbf{minimize}\qquad\sum_{e\in \mathcal{E}}c_{(e)}\cdot \color{blue}x_{(e)}$$

In [ ]:
mcf_LP.minimize(mcf_LP.sum(c[e]*x[e] for e in E))

## Summarize the model

In [ ]:
mcf_LP.print_information()

## Solve the LP and display the result

In [ ]:
mcf_sol = mcf_LP.solve()
if mcf_sol:
    mcf_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt

## For convenience, extract the solution to a dictionary named `sol_x`

In [ ]:
sol_x = mcf_sol.get_value_dict(x)
sol_x

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[S]['color'] = 'green'
g.vs[D]['color'] = 'red'

g.es['flow'] = [sol_x[e] for e in E]
g.es['cap'] = [mu[e] for e in E]
g.es['label'] = ' '
g.es['arrow_size'] = [10 if sol_x[e] > 0 else 6 for e in E]
g.es['arrow_width'] = [10 if sol_x[e] > 0 else None for e in E]
g.es['width'] = [3 if sol_x[e] > 0 else 0.5 for e in E]
g.es['color'] = ['darkblue' if sol_x[e] > 0 else None for e in E]
g.es['flow_label_color'] = ['blue' if sol_x[e] > 0 else 'black' for e in E]
g.es['cap_label_color']  = ['red' if sol_x[e] > 0 else 'black' for e in E]
g.es['label_size'] = [10 if sol_x[e] > 0 else 7 for e in E]

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

p = ig.plot(g, layout=g.layout('tree'), edge_label=g.es['label'], target=ax)

for e, edge_label in zip(g.es, p.get_edge_labels()):
    txt = ax.annotate(f'{e["flow"]:g}', xycoords=edge_label, xy=(-5,0), color=e['flow_label_color'], backgroundcolor='white', size=e['label_size'])
    txt =  ax.annotate(f' | ', xycoords=txt, xy=(1,0), color='k', size=e['label_size'], verticalalignment='bottom')
    txt =  ax.annotate(f'{e["cap"]}', xycoords=txt, xy=(1,0), color=e['cap_label_color'], backgroundcolor='white', size=e['label_size'], verticalalignment='bottom')
    
plt.show()